In [1]:
from datetime import datetime, timedelta
import pandas as pd
import psycopg2
from psycopg2.extras import RealDictCursor

In [2]:
POSTGRES = {
    'dbname': 'postgres',
    'user': 'postgres',
    'password': 'postgres',
    'host': '127.0.0.1',
    'port': 5432,
}

In [3]:
class Postgres:
    def __init__(self):
        try:
            self._conn = psycopg2.connect(**POSTGRES)
        except Exception as conn_error:
            print(conn_error)
            raise conn_error
        self._conn.set_client_encoding('UTF8')

    def execute(self, sql_give, args=None):
        if args is None:
            args = {}

        with self._conn.cursor(cursor_factory=RealDictCursor) as cursor:
            if isinstance(args, list):
                for i in args:
                    cursor.execute(sql_give, i)
            else:
                cursor.execute(sql_give, args)
        self._conn.commit()

    @classmethod
    def fetchall(cls, sql_give):
        pg = cls()
        try:
            return pg._fetchall(sql_give)
        except psycopg2.Error as fetch_error:
            print(fetch_error)
        finally:
            pg.close()

    def _fetchall(self, sql_give):
        with self._conn.cursor(cursor_factory=RealDictCursor) as cursor:
            cursor.execute(sql_give)
            return cursor.fetchall()

    def close(self):
        self._conn.close()

### Посмотреть общую схему таблиц в базе данных

In [5]:
get_schema_sql = """
SELECT table_schema, table_name
FROM information_schema.tables
ORDER BY table_schema,table_name;
"""
conn = Postgres()
data = pd.DataFrame(
     conn.fetchall(get_schema_sql)
     )
conn.close()

for table_schema in data.table_schema.unique():
    print(data[data['table_schema'] == table_schema].to_markdown(), end='\n \n')

|    | table_schema       | table_name                            |
|---:|:-------------------|:--------------------------------------|
|  0 | information_schema | _pg_foreign_data_wrappers             |
|  1 | information_schema | _pg_foreign_servers                   |
|  2 | information_schema | _pg_foreign_table_columns             |
|  3 | information_schema | _pg_foreign_tables                    |
|  4 | information_schema | _pg_user_mappings                     |
|  5 | information_schema | administrable_role_authorizations     |
|  6 | information_schema | applicable_roles                      |
|  7 | information_schema | attributes                            |
|  8 | information_schema | character_sets                        |
|  9 | information_schema | check_constraint_routine_usage        |
| 10 | information_schema | check_constraints                     |
| 11 | information_schema | collation_character_set_applicability |
| 12 | information_schema | collations          

### Создать таблицу

In [6]:
create_table_sql = """
CREATE TABLE public.example_table (
    id integer not null PRIMARY KEY,
    col1 text,
    col2 text,
    col3 text,
    col4 bool
);
CREATE unique index on public.example_table (id);
"""
conn = Postgres()
conn.execute(create_table_sql)
conn.close()

### Добавить запись в таблицу

In [7]:
def insert_test(element):
    if isinstance(element, str):
        return element.strip()
    return element

insert_into_sql = """
INSERT INTO public.example_table (
    id,
    col1,
    col2,
    col3,
    col4
) VALUES (%s,%s,%s,%s,%s);
"""

add = pd.DataFrame([
    [1, '1','A','a', False],
    [2, '2','B','b', True],
    [3, '3','C','c', False]
], columns = ['id','col1','col2','col3','col4'])

conn = Postgres()

for row in add.values:
    display(row)
    conn.execute(insert_into_sql,
                 tuple(insert_test(element) for element in row))
    
conn.close()

array([1, '1', 'A', 'a', False], dtype=object)

array([2, '2', 'B', 'b', True], dtype=object)

array([3, '3', 'C', 'c', False], dtype=object)

### Получить данные из таблицы

In [11]:
select_sql = """
SELECT * 
FROM public.example_table;

"""

conn = Postgres()
data = pd.DataFrame(
    conn.fetchall(select_sql)
    )
conn.close()

data

,id,col1,col2,col3,col4
0,1,1,A,a,False
1,2,2,B,b,True
2,3,3,C,c,False


### Изменить таблицу

In [10]:
alter_table_sql = """
ALTER TABLE public.example_table
ALTER COLUMN col1 TYPE integer USING col1::integer;
"""

conn = Postgres()
conn.execute(alter_table_sql)
conn.close()

### Удалить записи из таблицы

In [ ]:
delete_values_sql = """
DELETE from public.example_table
WHERE col4 = True;
"""

conn = Postgres()
conn.execute(delete_values_sql)
conn.close()

### Удалить таблицу

In [13]:
drop_table_sql = """DROP table public.example_table;"""

conn = Postgres()
conn.execute(drop_table_sql)
conn.close()